<a href="https://colab.research.google.com/github/RoboTuan/ML_DL_Project/blob/master/Ges%C3%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title
import torch
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn
import tensorflow as tf
import datetime
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from torch.autograd import Function
import torchvision
from torchvision.models import alexnet
from torchvision.utils import make_grid
from sklearn.model_selection import RandomizedSearchCV
import pandas.util.testing as tm
from PIL import Image
from tqdm.notebook import tqdm

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip -q "./drive/My Drive/Copia di GTEA61.zip"
!rm -rf "./__MACOSX"

In [4]:
!rm -rf "./ML_DL_Project/"
#!rm -rf "./out_dir/"

In [5]:
import os 
if not os.path.isdir('./ML_DL_Project'):
  !git clone https://github.com/RoboTuan/ML_DL_Project.git

Cloning into 'ML_DL_Project'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 366 (delta 92), reused 77 (delta 43), pack-reused 230
Receiving objects: 100% (366/366), 89.24 MiB | 14.92 MiB/s, done.
Resolving deltas: 100% (226/226), done.


In [6]:
base_dataset = "./GTEA61"

if not os.path.isdir('./GTEA61_val'):
  os.makedirs("./GTEA61_val")
  os.makedirs("./GTEA61_val/flow_x_processed")
  os.makedirs("./GTEA61_val/flow_y_processed")
  os.makedirs("./GTEA61_val/processed_frames2")
  pass


for directory in sorted(os.listdir(base_dataset)):
  if not directory.startswith('.'):
    directory_path = os.path.join(base_dataset, directory)
    new_directory_path = os.path.join("./GTEA61_val", directory)

    for user in sorted(os.listdir(directory_path)):
      if not user.startswith('.') and user == "S2":
        current_folder = os.path.join(directory_path, user)
        !cp -r "{current_folder}" "{new_directory_path}"
        !rm -rf "{current_folder}"
        print(current_folder)

./GTEA61/flow_x_processed/S2
./GTEA61/flow_y_processed/S2
./GTEA61/processed_frames2/S2


In [17]:
# Se scritti uguali ai parametri della funzione può venire None come risultato
# per questo SEQLEN è maiuscolo 
SEQLEN = 7
DEVICE = 'cuda' # 'cuda' or 'cpu'
num_classes = 61
#STAGE = None
train_data_dir = "./GTEA61"
val_data_dir = "./GTEA61_val"
#stage1_dict = None
outDir = ".out_dir"
trainBatchSize = 32
valBatchSize = 64
numEpochs1 = 3
numEpochs2 = 3
lr1 = 1e-3
lr2 = 1e-4
decay_factor = 0.1
decay_step1 = [25, 75, 150]
decay_step2 = [25, 75]
#Boh?!
memSize = 512
alpha=1

In [ ]:
#!rm -rf "ML_DL_Project"
#!rm -rf "./GTEA61"
#!rm -rf "./GTEA61_val"

In [8]:
from __future__ import print_function, division
# This is without attention, we must address this better
#from ML_DL_Project.Scripts.convLSTMmodel import *
from ML_DL_Project.Scripts.SelfSupObjectAttentionModelConvLSTM import *
#from ML_DL_Project.Scripts.RegObjectAttentionModelConvLSTM import *
from ML_DL_Project.Scripts.spatial_transforms import (Compose, ToTensor, CenterCrop, Scale, Normalize, MultiScaleCornerCrop,
                                RandomHorizontalFlip)
from torch.utils.tensorboard import SummaryWriter
from ML_DL_Project.Scripts.resnetMod import *
#from ML_DL_Project.Scripts.makeDatasetRGB import *
from ML_DL_Project.Scripts.makeMmaps import *
#Prende il makeDataset dell'ultimo script importato
import argparse
import sys

In [9]:
!rm -rf "./out_dir"

In [10]:
# Data loader
normalize = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
spatial_transform = Compose([Scale(256),
                             RandomHorizontalFlip(), 
                             MultiScaleCornerCrop([1, 0.875, 0.75, 0.65625], 224),
                             ToTensor(), 
                             normalize])

vid_seq_train = makeDataset(train_data_dir, spatial_transform=spatial_transform, seqLen=SEQLEN, fmt='.png')

train_loader = torch.utils.data.DataLoader(vid_seq_train, batch_size=trainBatchSize, shuffle=True, num_workers=4, pin_memory=True)

if val_data_dir is not None:
  vid_seq_val = makeDataset(val_data_dir,spatial_transform = Compose([Scale(256),
                                                                    CenterCrop(224),
                                                                    ToTensor(),
                                                                    normalize]),
                            seqLen=SEQLEN, fmt='.png')

  val_loader = torch.utils.data.DataLoader(vid_seq_val, batch_size=valBatchSize, shuffle=False, num_workers=2, pin_memory=True)
  valInstances = vid_seq_val.__len__()


trainInstances = vid_seq_train.__len__()

In [11]:
for seqImages, seqMaps, labels in train_loader:
  print(f"Batch size: {len(seqImages)}")
  #Make dataset restituisce una sequenza di immagini e di mappe
  print(f"Batch size: {len(seqMaps)}")
  print(f"Number of frames selected: {len(seqImages[0])}")
  print(f"Number of frames selected maps: {len(seqMaps[0])}")

  break

Batch size: 32
Batch size: 32
Number of frames selected: 7
Number of frames selected maps: 7


In [19]:

model_folder = os.path.join('./', outDir, "GTEA61", 'selfSup')
#model_folder = os.path.join('./', "out_dir", "./GTEA61", 'rgb')
if os.path.exists(model_folder):
    print('Dir {} exists!'.format(model_folder))
    sys.exit()
os.makedirs(model_folder)

# Log files
writer = SummaryWriter(model_folder)
train_log_loss = open((model_folder + '/train_log_loss.txt'), 'w')
train_log_acc = open((model_folder + '/train_log_acc.txt'), 'w')
val_log_loss = open((model_folder + '/val_log_loss.txt'), 'w')
val_log_acc = open((model_folder + '/val_log_acc.txt'), 'w')

In [13]:
train_params = []
model = attentionModel(num_classes=num_classes, mem_size=memSize)
model.train(False)
for params in model.parameters():
    params.requires_grad = False

for params in model.lstm_cell.parameters():
    params.requires_grad = True
    train_params += [params]

for params in model.classifier.parameters():
    params.requires_grad = True
    train_params += [params]

model.lstm_cell.train(True)

model.classifier.train(True)
model.cuda()

loss_fn = nn.CrossEntropyLoss()

optimizer_fn = torch.optim.Adam(train_params, lr=lr1, weight_decay=4e-5, eps=1e-4)

optim_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer_fn, milestones=decay_step1,
                                                           gamma=decay_factor)



Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


In [20]:
valDatasetDir = val_data_dir

train_iter = 0
min_accuracy = 0
for epoch in range(numEpochs1):
    epoch_loss = 0
    numCorrTrain = 0
    mmap_loss = 0
    trainSamples = 0
    iterPerEpoch = 0
    
    model.lstm_cell.train(True)
    model.classifier.train(True)
    
    
    #for i, (inputs, targets) in enumerate(train_loader):
    for inputs, inputMmap, targets in train_loader:
        train_iter += 1
        iterPerEpoch += 1
        optimizer_fn.zero_grad()

        inputMmap = inputMmap.cuda()
        
        inputVariable = Variable(inputs.permute(1, 0, 2, 3, 4).cuda())
        labelVariable = Variable(targets.cuda())
        trainSamples += inputs.size(0)
        
        output_label, _ , mmapPrediction = model(inputVariable)

        mmapPrediction = mmapPrediction.view(-1,2)         
        inputMmap = torch.reshape(inputMmap, (-1,)) #.long()
        inputMmap = torch.round(inputMmap).long() #making things black and white again
        
        
        #print(mmapPrediction.size())
        #print(inputMmap.size())
        #print(mmapPrediction)
        #print(inputMmap)
        #sys.exit()

        loss_reg = alpha*loss_fn(mmapPrediction,inputMmap)
        loss = loss_fn(output_label, labelVariable)

        #Weighting the loss of the ss task
        #by multiplying it by alpha
        total_loss = loss  + loss2
        total_loss.backward()
        
        optimizer_fn.step()
        _, predicted = torch.max(output_label.data, 1)
        numCorrTrain += (predicted == targets.cuda()).sum()
        mmap_loss += loss2.item()
        epoch_loss += loss.item()
    
    optim_scheduler.step()
    avg_mmap_loss = mmap_loss / iterPerEpoch
    avg_loss = epoch_loss/iterPerEpoch
    trainAccuracy = torch.true_divide(numCorrTrain , trainSamples) * 100
    
    writer.add_scalar('train/epoch_loss', avg_loss, epoch+1)
    writer.add_scalar('train/accuracy', trainAccuracy, epoch+1)
    train_log_loss.write('Training loss after {} epoch = {}\n'.format(epoch+1, avg_loss))
    train_log_acc.write('Training accuracy after {} epoch = {}\n'.format(epoch+1, trainAccuracy))

    print('Train: Epoch = {} | Loss = {} | Accuracy = {}'.format(epoch+1, avg_loss, trainAccuracy))
    print('Mmap loss after {} epoch = {} '.format(epoch + 1, avg_mmap_loss))

    
    if valDatasetDir is not None:
        model.train(False)
        val_loss_epoch = 0
        val_iter = 0
        val_mmap_loss = 0
        val_samples = 0
        numCorr = 0
        mmap_loss = 0

        with torch.no_grad():
            #for j, (inputs, targets) in enumerate(val_loader):
            for inputs, inputMmap, targets in val_loader:
                val_iter += 1
                val_samples += inputs.size(0)
                inputMmap = inputMmap.cuda()

                
                inputVariable = Variable(inputs.permute(1, 0, 2, 3, 4).cuda())
                labelVariable = Variable(targets.cuda(async=True))
                #labelVariable = Variable(targets.cuda())
                
                output_label, _ , mmapPrediction = model(inputVariable)

                mmapPrediction = mmapPrediction.view(-1,2)
                inputMmap = torch.reshape(inputMmap, (-1,)) #.long()
                inputMmap = torch.round(inputMmap).long()
                loss2 = alpha*loss_fn(mmapPrediction,inputMmap)

                val_loss = loss_fn(output_label, labelVariable)
                val_loss_epoch += val_loss.item()
                val_mmap_loss += loss2.item()
                
                _, predicted = torch.max(output_label.data, 1)
                numCorr += (predicted == targets.cuda()).sum()

        avg_mmap_val_loss = val_mmap_loss / val_iter
        val_accuracy = torch.true_divide(numCorr , val_samples) * 100
        avg_val_loss = val_loss_epoch / val_iter
        
        writer.add_scalar('val/epoch_loss', avg_val_loss, epoch + 1)
        writer.add_scalar('val/accuracy', val_accuracy, epoch + 1)
        val_log_loss.write('Val Loss after {} epochs = {}\n'.format(epoch + 1, avg_val_loss))
        val_log_acc.write('Val Accuracy after {} epochs = {}%\n'.format(epoch + 1, val_accuracy))

        print('Val MMap Loss after {} epochs, loss = {}'.format(epoch + 1, avg_mmap_val_loss))
        print('Val: Epoch = {} | Loss {} | Accuracy = {}'.format(epoch + 1, avg_val_loss, val_accuracy))

        if val_accuracy > min_accuracy:
            save_path_model = (model_folder + '/model_rgb_state_dict.pth')
            torch.save(model.state_dict(), save_path_model)
            min_accuracy = val_accuracy


Train: Epoch = 1 | Loss = 3.843002514405684 | Accuracy = 8.797654151916504
Mmap loss after 1 epoch = 0.8008179068565369 
Val MMap Loss after 1 epochs, loss = 0.8123058676719666
Val: Epoch = 1 | Loss 3.693465828895569 | Accuracy = 9.482758522033691
Train: Epoch = 2 | Loss = 3.7302194291895088 | Accuracy = 10.85044002532959
Mmap loss after 2 epoch = 0.8008179068565369 
Val MMap Loss after 2 epochs, loss = 0.8123058676719666
Val: Epoch = 2 | Loss 3.5182019472122192 | Accuracy = 12.068964958190918
Train: Epoch = 3 | Loss = 3.6424417278983374 | Accuracy = 9.970674514770508
Mmap loss after 3 epoch = 0.8008179068565369 
Val MMap Loss after 3 epochs, loss = 0.8123058676719666
Val: Epoch = 3 | Loss 3.5622751712799072 | Accuracy = 11.206896781921387
